
# Cup & Handle Scanner — No-Args Notebook (Daily, IST)
This notebook detects **Cup & Handle** patterns on daily data and filters the stocks that currently show a **Cup**, **Handle**, or **Breakout** — all **without passing variables** to functions.  
All settings live in a single **Configuration** cell as globals, and every function reads from those globals.

**Outputs**
- A tidy table of matches
- A CSV file: `cup_handle_scan_results.csv`
- Optional chart for the first detected pattern


In [1]:

# === 1) Dependencies ===
# If yfinance / pandas / numpy / matplotlib are not installed, install them.
# (Safe to re-run; skips if already present.)

import sys, subprocess

def _ensure(pkg):
    try:
        __import__(pkg)
    except Exception:
        print("Installing", pkg, "...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

for p in ["yfinance", "pandas", "numpy", "matplotlib"]:
    _ensure(p)

import pandas as pd, numpy as np
import yfinance as yf
import math, os
from dataclasses import dataclass
from typing import Optional, Tuple


In [2]:

# === 2) Configuration (Globals) ===
# Put your tickers list here OR create a file named 'tickers.txt' (one ticker per line).
# NOTE: All downstream code reads these globals; no variables are passed into functions.

TICKERS = [
    # Example universe — replace with your list or create 'tickers.txt'
    "RELIANCE.NS", "TCS.NS", "INFY.NS"
]

LOOKBACK_DAYS = 450   # ~1.5 years
MIN_CUP_LEN = 30
MAX_CUP_LEN = 200
MIN_DEPTH_PCT = 8.0
MAX_DEPTH_PCT = 55.0
HANDLE_MAX_DROP_PCT = 35.0
HANDLE_MAX_LEN = 30
RIM_TOLERANCE_PCT = 3.0
BREAKOUT_WINDOW = 10
VOL_MULT = 1.3
TZ = "Asia/Kolkata"
PLOT = True   # Plot the first detected pattern
OUT_CSV = "cup_handle_scan_results.csv"

# If 'tickers.txt' exists (one ticker per line), it overrides TICKERS above.
if os.path.exists("tickers.txt"):
    with open("tickers.txt", "r", encoding="utf-8") as f:
        TICKERS = [ln.strip() for ln in f if ln.strip() and not ln.startswith("#")]

print("Loaded", len(TICKERS), "tickers.")


Loaded 3 tickers.


In [3]:

# === 3) Core Detection Logic (reads globals; no function args) ===
import pandas as pd
import numpy as np
import math
from dataclasses import dataclass
from typing import Optional, Tuple

def _pct(a, b):
    # Return percent drop from a to b: (a-b)/a * 100
    if a == 0 or pd.isna(a) or pd.isna(b):
        return np.nan
    return (a - b) / a * 100.0

def _swing_highs_lows(series, left=3, right=3):
    # Identify swing highs and lows using a centered rolling window.
    highs = series.rolling(left + 1 + right, center=True).apply(lambda x: float(x.argmax() == left), raw=False)
    lows  = series.rolling(left + 1 + right, center=True).apply(lambda x: float(x.argmin() == left), raw=False)
    sh = np.where(highs == 1.0)[0]
    sl = np.where(lows == 1.0)[0]
    return sh, sl

@dataclass
class CupHandlePattern:
    left_rim_idx: int
    trough_idx: int
    right_rim_idx: int
    handle_start_idx: Optional[int]
    handle_low_idx: Optional[int]
    handle_end_idx: Optional[int]
    breakout_idx: Optional[int]
    label: str  # 'Cup', 'Handle', 'Breakout'

def _find_cup_handle(df):
    # Detect the most recent Cup/Handle/Breakout using global constraints.
    # Uses globals: MIN_CUP_LEN, MAX_CUP_LEN, MIN_DEPTH_PCT, MAX_DEPTH_PCT,
    #               HANDLE_MAX_DROP_PCT, HANDLE_MAX_LEN, RIM_TOLERANCE_PCT,
    #               BREAKOUT_WINDOW, VOL_MULT
    close = df["Close"].values
    high  = df["High"].values
    low   = df["Low"].values
    vol   = df["Volume"].values.astype(float)

    df["Close_s"] = df["Close"].rolling(3, center=True).median().fillna(df["Close"])

    swing_hi_idx, swing_lo_idx = _swing_highs_lows(df["Close_s"], left=3, right=3)

    N = len(df)
    for rr in reversed(swing_hi_idx):
        left_candidates = swing_hi_idx[swing_hi_idx < rr]
        if len(left_candidates) == 0:
            continue
        lr = left_candidates[-1]

        cup_len = rr - lr
        if cup_len < MIN_CUP_LEN or cup_len > MAX_CUP_LEN:
            continue

        mids = swing_lo_idx[(swing_lo_idx > lr) & (swing_lo_idx < rr)]
        if len(mids) == 0:
            continue
        trough = mids[np.argmin(low[mids])]

        left_rim_price  = high[lr]
        right_rim_price = high[rr]
        trough_price    = low[trough]

        rim_diff_pct = abs(left_rim_price - right_rim_price) / max(left_rim_price, right_rim_price) * 100.0
        if rim_diff_pct > RIM_TOLERANCE_PCT:
            continue

        depth_pct = _pct(left_rim_price, trough_price)
        if math.isnan(depth_pct) or depth_pct < MIN_DEPTH_PCT or depth_pct > MAX_DEPTH_PCT:
            continue

        handle_start = rr + 1
        if handle_start >= N - 1:
            return CupHandlePattern(lr, trough, rr, None, None, None, None, "Cup")

        handle_window_end = min(N - 1, rr + HANDLE_MAX_LEN)
        handle_low = handle_start + int(np.argmin(low[handle_start:handle_window_end+1]))
        handle_drop_pct = _pct(right_rim_price, low[handle_low])

        cup_height = (left_rim_price + right_rim_price) / 2.0 - trough_price
        half_cup_level = trough_price + 0.5 * cup_height

        valid_handle = (
            handle_drop_pct <= HANDLE_MAX_DROP_PCT and
            low[handle_low] >= half_cup_level
        )
        if not valid_handle:
            return CupHandlePattern(lr, trough, rr, None, None, None, None, "Cup")

        rim_level = max(left_rim_price, right_rim_price)
        vol_ma20 = pd.Series(vol).rolling(20).mean().values
        breakout_idx = None
        search_start = handle_low + 1
        search_end = min(N - 1, handle_low + BREAKOUT_WINDOW)

        for i in range(search_start, search_end + 1):
            if close[i] > rim_level and (not math.isnan(vol_ma20[i])) and vol[i] > VOL_MULT * vol_ma20[i]:
                breakout_idx = i
                break

        if breakout_idx is not None:
            return CupHandlePattern(lr, trough, rr, handle_start, handle_low, i, breakout_idx, "Breakout")
        else:
            return CupHandlePattern(lr, trough, rr, handle_start, handle_low, handle_window_end, None, "Handle")

    return None


In [4]:

# === 4) Scan Universe (no args; uses globals) ===
from datetime import datetime

def _scan_one(ticker):
    try:
        data = yf.download(ticker, period=f"{LOOKBACK_DAYS}d", interval="1d", auto_adjust=False, progress=False)
        if data is None or data.empty:
            return None

        if isinstance(data.index, pd.DatetimeIndex):
            if data.index.tz is None:
                data.index = data.index.tz_localize("UTC")
            data.index = data.index.tz_convert(TZ)

        pat = _find_cup_handle(data)
        if pat is None:
            return None

        def _dt(i):
            return data.index[i].isoformat() if i is not None else None

        left_r = float(data["High"].iloc[pat.left_rim_idx])
        right_r = float(data["High"].iloc[pat.right_rim_idx])
        trough  = float(data["Low"].iloc[pat.trough_idx])

        return {
            "Ticker": ticker,
            "Label": pat.label,
            "LeftRimDate": _dt(pat.left_rim_idx),
            "TroughDate": _dt(pat.trough_idx),
            "RightRimDate": _dt(pat.right_rim_idx),
            "HandleStart": _dt(pat.handle_start_idx),
            "HandleLow": _dt(pat.handle_low_idx),
            "HandleEnd": _dt(pat.handle_end_idx),
            "BreakoutDate": _dt(pat.breakout_idx),
            "LeftRimHigh": left_r,
            "RightRimHigh": right_r,
            "TroughLow": trough,
            "CupDepthPct": float(_pct(left_r, trough)),
            "RimLevel": float(max(left_r, right_r)),
            "LastClose": float(data["Close"].iloc[-1]),
            "LastDate": data.index[-1].isoformat(),
        }
    except Exception as e:
        return {"Ticker": ticker, "Label": "Error", "Error": str(e)}

rows = []
for t in TICKERS:
    r = _scan_one(t)
    if r is not None:
        rows.append(r)

if rows:
    res = pd.DataFrame(rows)
    rank = {"Breakout": 0, "Handle": 1, "Cup": 2, "Error": 3}
    res["Rank"] = res["Label"].map(rank).fillna(4)
    res = res.sort_values(["Rank","Ticker"]).drop(columns=["Rank"])
else:
    res = pd.DataFrame(columns=[
        "Ticker","Label","LeftRimDate","TroughDate","RightRimDate","HandleStart","HandleLow","HandleEnd",
        "BreakoutDate","LeftRimHigh","RightRimHigh","TroughLow","CupDepthPct","RimLevel","LastClose","LastDate","Error"
    ])

display(res)
res.to_csv(OUT_CSV, index=False)
print("Saved results to", OUT_CSV)


,Ticker,Label,LeftRimDate,TroughDate,RightRimDate,HandleStart,HandleLow,HandleEnd,BreakoutDate,LeftRimHigh,RightRimHigh,TroughLow,CupDepthPct,RimLevel,LastClose,LastDate,Error


Saved results to cup_handle_scan_results.csv


In [5]:

# === 5) Optional Plot (first detected pattern) ===
if PLOT and not res.empty:
    try:
        import matplotlib.pyplot as plt
        to_plot = res[res["Label"].isin(["Breakout","Handle","Cup"])]
        if not to_plot.empty:
            row = to_plot.iloc[0]
            tk = row["Ticker"]
            data = yf.download(tk, period=f"{LOOKBACK_DAYS}d", interval="1d", auto_adjust=False, progress=False, multi_level_index=False)
            if isinstance(data.index, pd.DatetimeIndex):
                if data.index.tz is None:
                    data.index = data.index.tz_localize("UTC")
                data.index = data.index.tz_convert(TZ)

            plt.figure(figsize=(12,6))
            plt.plot(data.index, data["Close"])
            def mark(dt, label):
                if dt:
                    x = pd.to_datetime(dt)
                    if x in data.index:
                        y = float(data.loc[x, "Close"])
                        plt.scatter([x], [y])
                        plt.text(x, y, label)

            mark(row["LeftRimDate"], "Left Rim")
            mark(row["TroughDate"], "Trough")
            mark(row["RightRimDate"], "Right Rim")
            mark(row["HandleLow"], "Handle Low")
            mark(row["BreakoutDate"], "Breakout")

            plt.title(f"{tk} — Cup & Handle ({row['Label']})")
            plt.xlabel("Date"); plt.ylabel("Close")
            plt.tight_layout()
            plt.show()
        else:
            print("No Cup/Handle/Breakout to plot.")
    except Exception as e:
        print("Plotting skipped:", e)
else:
    print("Plotting disabled or no results.")


Plotting disabled or no results.
